In [183]:
import numpy as np
import pandas as pd
import os
import csv
import regex as reg
import joblib

#import geopandas as gpd
from sklearn.preprocessing import LabelEncoder

In [184]:
directory = os.listdir('files/')
print("total dataset including ties and alternate stratego versions")
print(len(directory))

total dataset including ties and alternate stratego versions
51589


Number of barrage(special version of Stratego) files, run code to remove

In [185]:
num_barrage = 0
directory = os.listdir('files/')
print("before barrage files removal")
print(len(directory))
for file in directory:
    if file.endswith('.xml') and 'barrage' in file:
        num_barrage += 1
        os.remove(os.path.join('files/', file))
print("number of barrage files removed")
print(num_barrage)
directory = os.listdir('files/')
print(len(directory))


before barrage files removal
51589
number of barrage files removed
0
51589


Removes the Freesetup files

In [186]:
num_classic_free = 0
directory = os.listdir('files/')
print("before classic_free files removal")
print(len(directory))
for file in directory:
    if file.endswith('.xml') and 'classicfree' in file:
        num_classic_free += 1
        os.remove(os.path.join('files/', file))
print("number of classic_free files removed")
print(num_classic_free)
directory = os.listdir('files/')
print(len(directory))


before classic_free files removal
51589
number of classic_free files removed
0
51589


Removes the ultimate lightning files

In [187]:

num_ultimate_lightning = 0
directory = os.listdir('files/')
print("before ultimate_lightning files removal")
print(len(directory))
for file in directory:
    if file.endswith('.xml') and 'ultimate lightning' in file:
        num_ultimate_lightning += 1
        os.remove(os.path.join('files/', file))
print("number of ultimate_lightning files removed")
print(num_ultimate_lightning)
directory = os.listdir('files/')
print(len(directory))

before ultimate_lightning files removal
51589
number of ultimate_lightning files removed
256
51333


Removes the duell files

In [188]:

num_classic_free = 0
directory = os.listdir('files/')
print("before duell files removal")
print(len(directory))
for file in directory:
    if file.endswith('.xml') and 'duell' in file:
        num_classic_free += 1
        os.remove(os.path.join('files/', file))
print("number of duell files removed")
print(num_classic_free)
directory = os.listdir('files/')
print(len(directory))

before duell files removal
51333
number of duell files removed
0
51333


How many ties are in the dataset, and removed from directory list

In [189]:
directory = os.listdir('files/')

total_tie = 0
gsn_files = [file for file in directory if file.endswith('.gsn')]
xml_files = [file for file in directory if file.endswith('.xml')]

gsn_indexes = []
xml_indexes = []

for index, gsn_file in enumerate(gsn_files):
    with open('files/' + str(gsn_file), 'r') as file:
        gsn_data = file.read()
        #print(gsn_file)
    if int(gsn_data.split('\n')[-2][-1]) == 2:
        total_tie +=1
        gsn_indexes.append(index)
        file.close()

for index, xml_file in enumerate(xml_files):
    with open('files/' + str(xml_file), 'r') as file:
        xml_data = file.read()
    if int(xml_data.split()[-3].split('"')[1]) == 3:
        total_tie +=1
        xml_indexes.append(index)
        file.close()
        continue


In [190]:
gsn_files = [file for index, file in enumerate(gsn_files) if index not in gsn_indexes]
xml_files = [file for index, file in enumerate(xml_files) if index not in xml_indexes]
print(len(gsn_files)),print(len(xml_files)), print(len(gsn_files)+len(xml_files))

8615
40952
49567


(None, None, None)

In [191]:
print('len dataset')
print(len(os.listdir('files/')))
print('dataset - ties')
print(len(gsn_files)+len(xml_files))
print('proportion of ties')
print(1-(len(gsn_files)+len(xml_files))/(len(os.listdir('files/'))))
print('total amount of ties')
print(total_tie)

len dataset
51333
dataset - ties
49567
proportion of ties
0.03440282079753765
total amount of ties
1766


Number of matches in dataset

In [192]:
print("Number of matches in Strados")
print(len(gsn_files)+len(xml_files))
print("Number of matches after getting both sides:")
print((len(gsn_files)+len(xml_files))*2)


Number of matches in Strados
49567
Number of matches after getting both sides:
99134


Extract GSN data and puts it in 2 arrays; 1 for wins and 1 for positions

In [193]:
winner_list_gsn = np.array([])
all_encoded_data_1 = np.array([])
all_encoded_data_2 = np.array([])
gsn_game_dataframe = pd.DataFrame()
all_encoded_data_gsn = np.array([])
both_player_encoded_data = np.array([])
player_list_1_gsn = []
player_list_2_gsn = []
total_pos_list_gsn = list()

for gsn_file in gsn_files:
    with open('files/' + str(gsn_file), 'r') as file:
        gsn_data = file.read()
    if int(gsn_data.split('\n')[-2][-1]) == 2: #check if it is not a tie
        #total matches including ties was 85292 without its 82216
        continue
    #elif int(xml_data.split()[-3].split('"')[1]) == 1:
    #    continue
    else:
        player_1_side_str = gsn_data.split('\n')[2][0:40]
        ##split the starting positions per player
        player_2_side_str = gsn_data.split('\n')[2][60:100]
        
        player_1_side = np.array(list(player_1_side_str)) 
        player_2_side = np.array(list(player_2_side_str))#creates arrays for encoding the starting positions
        #print(gsn_file)

        unique_pieces_1 = np.unique(player_1_side)
        unique_pieces_2 = np.unique(player_2_side)
        #creates an identity matrix with shape (number of letters, number of letters)
        identity_matrix = np.eye(len(unique_pieces_1))
        identity_matrix = np.eye(len(unique_pieces_2))
        #maps letter values to dummy variables
        dummy_variables_1 = identity_matrix[np.searchsorted(unique_pieces_1, player_1_side)]
        dummy_variables_1 = dummy_variables_1.reshape(4,10,12)
        dummy_variables_2 = identity_matrix[np.searchsorted(unique_pieces_2, player_2_side)]
        dummy_variables_2 = dummy_variables_2.reshape(4,10,12)        


        player_list_1_gsn.append(dummy_variables_1)
        player_list_2_gsn.append(dummy_variables_2)
        winner_list_gsn = np.append(winner_list_gsn, int(gsn_data.split('\n')[-2][-1])) #creates a list of who won which game 1 indicates player 2 and 0 is the red player(player 1)

player_list_1_gsn.extend(player_list_2_gsn)#creates a total list with the first players position being the first players'(red) arrays
inverted_winner_list = 1 - winner_list_gsn
both_sided_winner_list_gsn = np.append(inverted_winner_list, winner_list_gsn) # creates double the size of the winner list # doubles the size of the winner list by inverting the list

In [205]:
len(both_sided_winner_list_gsn),len(player_list_1_gsn), dummy_variables_2.shape


17230

'<?xml version="1.0" encoding="UTF-8"?>\n<stratego>\n <game type="classic">\n  \n  <player id="1">Player 1</player>\n  <player id="2">Player 2</player>\n  <result type="3" winner="1"/>\n </game>\n</stratego>'

Extract XML data and puts it in 2 arrays; 1 for wins and 1 for positions

In [231]:
winner_list_xml = np.array([])
all_encoded_data_1 = np.array([])
all_encoded_data_2 = np.array([])
xml_game_dataframe = pd.DataFrame()
all_encoded_data_xml = np.array([])
both_player_encoded_data = np.array([])
player_list_1_xml = []
player_list_2_xml = []
num_invalid = 0

for xml_file in xml_files:
    with open('files/' + str(xml_file), 'r') as file:
        xml_data = file.read()

    if int(xml_data.split()[-3].split('"')[1]) == 3: #check if it is not a tie
        #total matches including ties was 85292 without its 82216
        continue
    if len(xml_data.split()[7].split('"')[1][0:40])!=40: #check if it is not an invalid match(does not contain deployment information)
        num_invalid += 1
        print(xml_file) #exactly 1 file has no deployment for some reason?
        continue
    else:


        #print(xml_file)
        player_1_side_str = xml_data.split()[7].split('"')[1][0:40]
        ##split the starting positions per player
        player_2_side_str = xml_data.split()[7].split('"')[1][60:100]
        
        player_1_side = np.array(list(player_1_side_str)) 
        player_2_side = np.array(list(player_2_side_str)) #create arrays for encoding the starting positions
        
        unique_pieces_1 = np.unique(player_1_side)
        unique_pieces_2 = np.unique(player_2_side)
        # Create an identity matrix with shape (number of letters, number of letters)
        identity_matrix = np.eye(len(unique_pieces_1))
        identity_matrix = np.eye(len(unique_pieces_2))
        # Map letter values to dummy variables
        dummy_variables_1 = identity_matrix[np.searchsorted(unique_pieces_1, player_1_side)]
        dummy_variables_1 = dummy_variables_1.reshape(4,10,12)
        dummy_variables_2 = identity_matrix[np.searchsorted(unique_pieces_2, player_2_side)]
        dummy_variables_2 = dummy_variables_2.reshape(4,10,12)        


        player_list_1_xml.append(dummy_variables_1)
        player_list_2_xml.append(dummy_variables_2)
        #encoder = LabelEncoder()
        #encoded_data_1 = encoder.fit_transform(list(player_1_side)) #create dummy variables out of letters
        #encoded_data_2 = encoder.fit_transform(list(player_2_side))
        #if len(all_encoded_data_1) == 0:
        #    all_encoded_data_1 = encoded_data_1
        #    all_encoded_data_2 = encoded_data_2
        #else: 
        #    all_encoded_data_1 = np.vstack((all_encoded_data_1, encoded_data_1)) #add encoding to long array of all encoding for all games
        #    all_encoded_data_2 = np.vstack((all_encoded_data_2, encoded_data_2))
        winner_list_xml = np.append(winner_list_xml, int(xml_data.split()[-3].split('"')[1])) #creates a list of who won which game 1 indicates player 2 and 0 is the red player(player 1)

player_list_1_xml.extend(player_list_2_xml)#creates a total list with the first players position being the first 9021 arrays
player_list_1_xml.extend(player_list_1_gsn)#combines all player positions from both file types
i=-1
for num in winner_list_xml:
    i+=1
    if num ==  2: ##means player 1 lost we change it to 0
        winner_list_xml[i] = 0    
inverted_winner_list_xml = 1 - winner_list_xml
both_sided_winner_list_xml = np.append(winner_list_xml, inverted_winner_list_xml) # creates double the size of the winner list # doubles the size of the winner list by inverting the list
all_winner_list = np.concatenate((both_sided_winner_list_xml, both_sided_winner_list_gsn))


classic-2014.9-2525.xml


In [226]:
print(num_invalid)

1


In [230]:
len(player_list_1_xml), len(all_winner_list) #total length of xml data collected with the 1st array being independent and the 2nd dependent

(99132, 99132)

In [238]:
print(len(player_list_1_gsn)) 
print(len(player_list_1_xml)-len(player_list_1_gsn))

17230
81902


Writes the data to a csv file and combines the GSN and XML files

In [235]:
np.savetxt("Y_winners_encoded.csv", all_winner_list, delimiter=",")
joblib.dump(player_list_1_xml, "X_encoded_positions.txt")


['X_encoded_positions.txt']

In [236]:
joblib.load("X_encoded_positions.txt")

[array([[[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
        [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,